In [1]:
import numpy as np
import pandas as pd
import time

# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
# from sklearn.metrics import accuracy_score,confusion_matrix




In [2]:
trainSet = pd.read_csv("./fashion-mnist_train.csv")
# load train data and copy them
df_train = trainSet.copy()


In [3]:
# separating data and label
X_train = df_train.drop(['label'],axis = 1)
X_test = df_train['label']
X_test

# do nomalization
X_train = X_train.astype('float32')
X_train /= 255.0


In [4]:
#split train&test
seed = 99
np.random.seed(seed)
X_train, X_val,Y_train,Y_val = train_test_split(X_train,X_test,test_size =0.1, random_state= seed)
# do standard scaling
X_train=StandardScaler().fit_transform(X_train)
X_val=StandardScaler().fit_transform(X_val)

In [5]:
# PCA -> dimenstion Reducing
pca = PCA(n_components=100, random_state=42)
X_train_pca =pca.fit_transform(X_train)
X_test_pca = pca.transform(X_val)
X_train_PCA1 = pd.DataFrame(X_train_pca)
X_test_PCA1 = pd.DataFrame(X_test_pca)
X_train_pca.shape

(54000, 100)

In [6]:
# svm model
start = time.time()
# 
svc = SVC(C=1,kernel='rbf',gamma="auto",probability = True)
# Cost -->  13 / kernel --> gaussian /  gamma --> 1/# features
svc.fit(X_train_PCA1,Y_train)

end = time.time()
svm_time = end-start
print("SVM Time: {:0.2f} minute".format(svm_time/60.0))

SVM Time: 22.10 minute


In [7]:
# 혼성모델
start1 = time.time()

random_forest = RandomForestClassifier(criterion='entropy', max_depth=70, n_estimators=100)
random_forest.fit(X_train_PCA1, Y_train)

end1 = time.time()
forest_time = end1-start1
print("Random Forest Time: {:0.2f} minute".format(forest_time/60.0))

Random Forest Time: 5.03 minute


In [8]:
# 정확도 평가
y_train_svc = svc.predict(X_train_PCA1)
y_pred_svc = svc.predict(X_test_pca)
svc_train = metrics.accuracy_score(Y_train,y_train_svc)
svc_accuracy = metrics.accuracy_score(Y_val, y_pred_svc)

print("Train Accuracy score: {}".format(svc_train))
print("Test Accuracy score: {}".format(svc_accuracy))
print(metrics.classification_report(Y_val, y_pred_svc))

y_train_forest = random_forest.predict(X_train_PCA1)
y_pred_forest = random_forest.predict(X_test_pca)
random_forest_train = metrics.accuracy_score(Y_train,y_train_forest)
random_forest_accuracy = metrics.accuracy_score(Y_val, y_pred_forest)

print("Train Accuracy score: {}".format(random_forest_train))
print("Test Accuracy score: {}".format(random_forest_accuracy))
print(metrics.classification_report(Y_val, y_pred_forest))

Train Accuracy score: 0.9651296296296297
Test Accuracy score: 0.874
              precision    recall  f1-score   support

           0       0.84      0.81      0.82       574
           1       1.00      0.97      0.99       596
           2       0.83      0.80      0.82       573
           3       0.93      0.91      0.92       624
           4       0.85      0.84      0.85       635
           5       0.98      0.89      0.94       632
           6       0.77      0.66      0.71       600
           7       0.94      0.95      0.95       604
           8       0.71      0.99      0.83       598
           9       0.95      0.90      0.92       564

    accuracy                           0.87      6000
   macro avg       0.88      0.87      0.87      6000
weighted avg       0.88      0.87      0.87      6000

Train Accuracy score: 1.0
Test Accuracy score: 0.8723333333333333
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       57